In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import joblib

In [2]:
# importing Validation Dataset
df_valid = pd.read_csv('insurance_data_validation_set.csv')
df_valid

,age,gender,bmi,children,smoker,medical_history,family_medical_history,exercise_frequency,health_risk_score,smoker_exercise_interaction,...,coverage_level_2,coverage_level_3,bmi_category_1,bmi_category_2,bmi_category_3,age_group_1,age_group_2,children_category_1,children_category_2,charges
0,1.624282,-0.999786,1.647389,1.464043,-0.999742,0.446792,1.341704,-0.448141,1.749226,-0.808920,...,1.413658,-0.707396,-0.499142,-0.430922,0.769201,-0.674275,1.183895,-0.707148,1.000022,22463.419760
1,-0.829674,1.000214,-1.343401,1.464043,-0.999742,1.341609,1.341704,-1.343213,-1.176563,-1.270720,...,1.413658,-0.707396,2.003438,-0.430922,-1.300051,-0.674275,-0.844670,-0.707148,1.000022,22472.114558
2,-1.695776,-0.999786,1.675553,-1.463910,1.000258,-1.342842,1.341704,0.446931,1.669364,1.038281,...,-0.707385,1.413636,-0.499142,-0.430922,0.769201,-0.674275,-0.844670,-0.707148,-0.999978,14219.322834
3,0.397304,1.000214,-0.740043,0.878453,-0.999742,0.446792,1.341704,1.342003,-0.629394,0.114681,...,-0.707385,1.413636,-0.499142,2.320605,-1.300051,-0.674275,1.183895,-0.707148,1.000022,20500.098949
4,-1.695776,-0.999786,1.167520,-0.292728,-0.999742,0.446792,-1.340636,1.342003,1.109244,0.114681,...,-0.707385,-0.707396,-0.499142,-0.430922,0.769201,-0.674275,-0.844670,1.414132,-0.999978,20850.682981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1.263406,-0.999786,1.183768,1.464043,-0.999742,0.446792,1.341704,-1.343213,1.287317,-1.270720,...,-0.707385,-0.707396,-0.499142,-0.430922,0.769201,-0.674275,1.183895,-0.707148,1.000022,23892.584923
149996,-0.180097,-0.999786,1.605143,0.878453,1.000258,-1.342842,0.447590,-1.343213,1.545252,-0.808920,...,-0.707385,1.413636,-0.499142,-0.430922,0.769201,1.483074,-0.844670,-0.707148,1.000022,11980.947757
149997,1.119056,-0.999786,-0.969687,0.292862,1.000258,-0.448025,-0.446523,1.342003,-1.020075,1.961882,...,1.413658,-0.707396,-0.499142,2.320605,-1.300051,-0.674275,1.183895,-0.707148,1.000022,13178.228333
149998,0.902530,-0.999786,-0.958855,1.464043,-0.999742,0.446792,-0.446523,1.342003,-0.955321,0.114681,...,1.413658,-0.707396,-0.499142,2.320605,-1.300051,-0.674275,1.183895,-0.707148,1.000022,19493.004745


In [3]:
# Separate features and target from the validation set
X_val = df_valid.drop('charges', axis=1)
y_val = df_valid['charges']

rf_model = joblib.load('random_forest_model.joblib')

# Make predictions on the validation set
y_pred_val = rf_model.predict(X_val)

# Print the predictions
print("Predictions on the validation set:")
print(y_pred_val)

df_valid['charge_predict'] = y_pred_val


C:\Users\devar\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Predictions on the validation set:
[22237.02133523 22355.74410964 14219.94266288 ... 13111.32986353
 19941.19638746 18468.73589923]


In [4]:
df_valid[['charges', 'charge_predict']]

,charges,charge_predict
0,22463.419760,22237.021335
1,22472.114558,22355.744110
2,14219.322834,14219.942663
3,20500.098949,20705.518723
4,20850.682981,20954.815523
...,...,...
149995,23892.584923,23795.053436
149996,11980.947757,11968.268829
149997,13178.228333,13111.329864
149998,19493.004745,19941.196387


In [6]:
# Database connection details
host = r'127.0.0.1'
db = r'MSDS610' 
user = r'postgres' 
pw = r'8751' 
port = r'5432' 
schema = r'analytics'

# Create a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{db}')

# Create schema if it does not exist
with engine.connect() as connection:
    connection.execute(text("CREATE SCHEMA IF NOT EXISTS analytics"))

# Store the predictions in the 'predictions' table within the 'analytics' schema
df_valid.to_sql('predictions', con=engine, schema='analytics', if_exists='replace', index=False)

print("Predictions have been stored in the database.")

OperationalError: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)